# PYNQ Microblaze Libraries in C

This document describes the various libraries that ship with PYNQ Microblaze.

## `pynqmb`

The main library is `pynqmb` which consists of functions for interacting with a variety of I/O devices. `pynqmb` is split into separate `i2c.h`, `gpio.h`, `spi.h`, `timer.h` and `uart.h` header files with each one being self contained. In this notebook we will look just at the I2C and GPIO headers however the full function reference for all of the components can be found on http://pynq.readthedocs.io 

All of the components follow the same pattern in having `_open` function calls that take one or more pins depending on the protocol. These function use an I/O switch in the subsystem to connect the protocol controller to the output pins. For devices not connected to output pins there are `_open_device` functions which take either the base address of the controller or it's index as defined in the board support package.

For this example we are going to use a Grove ADC connected via Pmod-Grove adapter and using the I2C protocol. One ancilliary header file that is useful when using the Pmod-Grove adapter is `pmod_grove.h` which includes the pin definitions for the adapter board. In this case we are using the G4 port on the adapter which is connected to pins 6 and 2 of the Pmod connector.

In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay('base.bit')

In [2]:
%%microblaze base.PMODA
#include <i2c.h>
#include <pmod_grove.h>

int read_adc() {
    i2c device = i2c_open(PMOD_G4_B, PMOD_G4_A);
    unsigned char buf[2];
    buf[0] = 0;
    i2c_write(device, 0x50, buf, 1);
    i2c_read(device, 0x50, buf, 2);
    return ((buf[0] & 0x0F) << 8) | buf[1];
}

In [3]:
read_adc()

833

We can use the `gpio` and `timer` components in concert to flash an LED connected to G1. The `timer` header provides PWM and program delay functionality, although only one can be used simultaneously.

In [4]:
%%microblaze base.PMODA
#include <timer.h>
#include <gpio.h>
#include <pmod_grove.h>

void flash_led() {
    gpio led = gpio_open(PMOD_G1_A);
    gpio_set_direction(led, GPIO_OUT);
    int state = 0;
    while (1) {
        gpio_write(led, state);
        state = !state;
        delay_ms(500);
    }
}

In [5]:
flash_led()

## `pyprintf`

The `pyprint` library exposes a single `pyprintf` function which acts similarly to a regular `printf` function but forwards arguments to Python for formatting and display result in far lower code overhead than a regular printf as well as not requiring access to standard in and out.

In [6]:
%%microblaze base.PMODA
#include <pyprintf.h>

int test_print(float value) {
    pyprintf("Printing %f from the microblaze!\n", value);
    return 0;
}

In [7]:
test_print(1.5)

Printing 1.500000 from the microblaze!


0

At present, `pyprintf` can support the common subset of datatype between Python and C - in particular `%{douxXfFgGeEsc}`. Long data types and additional format modifiers are not supported yet.